# LangChain调用外部工具

In [ ]:
import pandas as pd

from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_experimental.tools import PythonAstREPLTool # 从LangChain依赖库引入Python代码解释器
from langchain_core.output_parsers.openai_tools import JsonOutputKeyToolsParser

df = pd.read_csv('global_cities_data.csv')
tool = PythonAstREPLTool(locals={"df": df}) # 传递给代码解释器的局部变量，这里是读取表格内容的pandas对象

# 使用 硅基流动 模型
model = init_chat_model(
    model="Qwen/Qwen3-8B",
    model_provider="openai",
    base_url="https://api.siliconflow.cn/v1/",
    api_key="sk-vjklrvkomqgjluwtfxssjnmetzvtwgfqgdgcyiycymhpjpzd",
)

# 添加提示词模板
system = f"""
你可以访问一个名为 `df` 的 pandas 数据框，你可以使用df.head().to_markdown() 查看数据集的基本信息， \
请根据用户提出的问题，编写 Python 代码来回答。只返回代码，不返回其他内容。只允许使用 pandas 和内置库。
"""

prompt = ChatPromptTemplate([
    ("system", system),
    ("user", "{question}")
])


parser = JsonOutputKeyToolsParser(key_name=tool.name, first_tool_only=True)

llm_with_tools = model.bind_tools([tool]) # 将工具与大模型绑定

llm_chain = prompt | llm_with_tools | parser | tool

response = llm_chain.invoke(
    "我有一张表，名为'df'，请帮我计算GDP_Billion_USD字段的均值。"
)

print(response)

但这事实上是用 “funtion calling” 实现的 （由于底层技术限制，大模型本身是无法和外部工具直接通信的），就是创建一个外部函数（function）作为中介，一边传递大模型的请求，另一边调用外部工具，最终让大模型能够间接的调用外部工具。

除了使用LangChain的内部工具，我们还可以自行创建外部函数并将其封装为一个LangChain“链”可调用的tool组件。

In [ ]:
# 首先引入tool装饰器，让LangChain能够识别外部函数，
# 注意函数介绍注释必须完整，装饰器通过读取注释提炼识别函数的内容

import requests
from langchain_core.tools import tool

@tool
def get_weather(city: str) -> str:
    """
    获取指定城市的当前天气信息。  
    :参数city: 城市名称，例如 "Beijing"。
    :返回: 包含天气描述的字符串，例如 "晴朗，25摄氏度"。
    """
    weather_url = ""
    weather_api_key = ""

    params = {
    "key": weather_api_key,
    "location": city,
    "language": "zh-Hans",
    "unit": "c",
}

    response = requests.get(weather_url, params=params).json()
    weather_data = response['results'][0]['now']

    return weather_data

# 将自定义函数注册为工具
tool = [get_weather] # 或者 tool = get_weather.to_tool()
# 将工具与大模型绑定
model = model.bind_tools(tool)

# 添加输出解析器
from langchain_core.output_parsers.openai_tools import JsonOutputKeyToolsParser
parser = JsonOutputKeyToolsParser(key_name=get_weather.name, first_tool_only=True)
# 构建链式调用
llm_chain = model | parser
get_weather_chain = llm_chain | get_weather

# 调用链式模型获取天气信息
response = get_weather_chain.invoke("请问上海今天天气如何?")
print(response)